# Stock Related Tweets Cleaning

In [1]:
import numpy as np
import pandas as pd
import spacy
import csv

In [2]:
nlp = spacy.load("en_core_web_lg")

In [41]:
df = pd.DataFrame(columns=["month", "day", "Title + Snippet", "Reddit Score", "Mentioned Companies"])

for index in range(2, 9):
    filename = "rawdata/RedditMentions{}.csv".format(index)
    temp_df = pd.read_csv(filename, header=0, usecols = ["Date", "Title + Snippet", "Reddit Score", "Mentioned Companies"], encoding="latin1")
    temp_df = temp_df.assign(month=pd.to_datetime(temp_df["Date"]).dt.month, day=pd.to_datetime(temp_df["Date"]).dt.day)
    temp_df = temp_df[["month", "day", "Title + Snippet", "Reddit Score", "Mentioned Companies"]]
    df = pd.concat([df, temp_df], ignore_index=True)

In [42]:
df.shape

(35000, 5)

In [44]:
comp_dict = {
    "TSLA": "tesla",
    "AAPL": "apple",
    "AMZN": "amazon",
    "NFLX": "netflex",
    "FB": "facebook",
    "GOOG": "google"
}

In [45]:
df

,month,day,Title + Snippet,Reddit Score,Mentioned Companies
0,8,10,"$TSLA Daily Investor Discussion - August 10, 2...",2,TSLA
1,5,6,This scale makes no sense (am I missing someth...,5,FB
2,9,21,Found this gem on fb. I will not rest until th...,21,FB
3,9,25,"FB account disabled, went to Oculus support fo...",1,FB
4,7,11,Full sent it and Risked $60K(pretty much my wh...,2,TSLA
...,...,...,...,...,...
34995,7,5,July 4th Fight/Driver Hits Individual. I am sh...,50,FB
34996,10,12,AAPL cash secured put with the goal to be assi...,3,AAPL
34997,7,3,"$300k on AMZN $3100c 8/21Next stop, Tendie Town!",0,AMZN
34998,8,21,"Friday August 21, 2020 Daily /r/thetagang Disc...",0,TSLA


In [46]:
selected_company = ["FB", "AMZN", "TSLA", "NFLX", "AAPL", "GOOG"]
for comp in selected_company:
    df_cp = df.loc[df["Mentioned Companies"].str.contains(comp, na=False)]
    for month in df_cp["month"].unique():
        df_mo = df_cp.loc[df_cp["month"]==month]
        for day in df_mo["day"].unique():
            df_final = df_mo.loc[df_mo["day"]==day]
            words = np.array([nlp(d).vector for d in df_final["Title + Snippet"]])
            np.savez("Reddit_cleaned/comment_vectors/{}-{}-{}".format(comp_dict[comp], month, day), inputs=words)
            np.savetxt("Reddit_cleaned/impact_weights/{}-{}-{}.txt".format(comp_dict[comp], month, day), df_final["Reddit Score"].to_numpy())